## Homework

This week we are going to use a new data set which contains 1070 purchases where the customer either purchased Citrus Hill or Minute Maid Orange Juice. 

In [1]:
import pandas as pd
oj = pd.read_csv('data/OJ.csv')
oj.head()

,Purchase,WeekofPurchase,StoreID,PriceCH,PriceMM,DiscCH,DiscMM,SpecialCH,SpecialMM,LoyalCH,SalePriceMM,SalePriceCH,PriceDiff,Store7,PctDiscMM,PctDiscCH,ListPriceDiff,STORE
0,0,237,1,1.75,1.99,0.00,0.0,0,0,0.500000,1.99,1.75,0.24,0,0.000000,0.000000,0.24,1
1,0,239,1,1.75,1.99,0.00,0.3,0,1,0.600000,1.69,1.75,-0.06,0,0.150754,0.000000,0.24,1
2,0,245,1,1.86,2.09,0.17,0.0,0,0,0.680000,2.09,1.69,0.40,0,0.000000,0.091398,0.23,1
3,1,227,1,1.69,1.69,0.00,0.0,0,0,0.400000,1.69,1.69,0.00,0,0.000000,0.000000,0.00,1
4,0,228,7,1.69,1.69,0.00,0.0,0,0,0.956535,1.69,1.69,0.00,1,0.000000,0.000000,0.00,0


In [2]:
data = oj.iloc[:, 1:]
target = oj.iloc[:, 0]

The features and the target variable are already prepared for you. What you are going to do is:

**1. Split**

Split the data set into two parts: training set and test set(with *random_state=0*, and *test_size=1.0/2*).


In [3]:
### your solution
from sklearn import model_selection as ms
x_train, x_test, y_train, y_test = ms.train_test_split(data, target, test_size=1.0/2, random_state=0)

**2. Support vector machine**

(1) Fit a svm model on the training set, report the training error and test error.(Just use the **svm.SVC** with default setting.)
    
(2) Change the value of parameter $C$ from $10^{-3}$ to $10^3$, make a plot to watch how the training error and test error varies. You can choose the value of $C$ from the array `np.logspace(-3, 3, 300)`.

In [9]:
### your solution
import numpy as np

from sklearn import svm
svm_model = svm.SVC()

svm_model.fit(x_train, y_train)
print(svm_model.score(x_train, y_train))
print(svm_model.score(x_test, y_test))

train_scores = []
test_scores = []
c_values = []

for i in np.logspace(start=-3, stop=3, base=10):
    svm_model.set_params(C=i)
    svm_model.fit(x_train, y_train)
        
    c_values.append(i)
    train_scores.append(svm_model.score(x_train, y_test))
    test_scores.append(svm_model.score(x_test, y_test))
        
import matplotlib.pyplot as plt
plt.scatter(c_values, train_scores)
plt.scatter(c_values, test_scores)
plt.show

0.7738317757009345
0.697196261682243


<function matplotlib.pyplot.show>

**3. Decision tree**

(1) Fit a decision tree model on the training set with the default setting.
    
(2) Set the depth of the tree from 1 to 30. Look the varies of the training error and test error.
    
(3) Use the function **grid_search.GridSearchCV** to find the best parameters. What's the best parameters? What's the best score? What's the training error and test error of the best model. The possible combination of the parameters may be:

In [4]:
grid_para_tree = {'criterion': ['gini', 'entropy'], 'max_depth': range(1, 31)}

(4) What are the first 5 important features?

In [6]:
### your solution
import numpy as np
from sklearn import tree
tree_model = tree.DecisionTreeClassifier()

grid_search = ms.GridSearchCV(tree_model, param_grid=grid_para_tree, scoring='accuracy', cv=3, n_jobs=1)
grid_search.fit(x_train, y_train)
print(grid_search.best_params_)
tree_model.set_params(criterion="gini", max_depth=4)
tree_model.fit(x_train, y_train)
feature_importance = np.array(list(zip(data.columns.values, tree_model.feature_importances_)), 
                              dtype=[('feature', 'S10'), ('importance', 'float')])
most_important = np.sort(feature_importance, order="importance")[::-1]
for i in most_important[0:5]:
    print(i)

{'criterion': 'gini', 'max_depth': 4}
(b'LoyalCH', 0.73021236)
(b'PriceDiff', 0.12488704)
(b'ListPriceD', 0.06672015)
(b'STORE', 0.04532273)
(b'WeekofPurc', 0.01607794)


**4. Random Forest.**

(1) Fit a random forest on the training set. Report the training error and test error.
    
(2) Use the function **grid_search.GridSearchCV** to find the best parameters. What's the best parameters? What's the best score? What's the training error and test error of the best model. The possible combination of the parameters may be:

In [8]:
grid_para_forest = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 31),
    'n_estimators': range(10, 110, 10)
}

(3) What's the first 5 important features?

In [9]:
### your solution
from sklearn import ensemble
forest_model = ensemble.RandomForestClassifier()

grid_rf_search = ms.GridSearchCV(forest_model, grid_para_forest, cv=3, n_jobs=1)
grid_rf_search.fit(x_train, y_train)
print(grid_rf_search.best_params_)
forest_model.set_params(criterion='gini', max_depth=3, n_estimators=80)
forest_model.fit(x_train, y_train)
print(forest_model.score(x_test, y_test))
feature_importance = np.array(list(zip(data.columns.values, forest_model.feature_importances_)), 
                              dtype=[('feature', 'S10'), ('importance', 'float')])
most_important = np.sort(feature_importance, order="importance")[::-1]
for i in most_important[0:5]:
    print(i)

{'criterion': 'gini', 'max_depth': 3, 'n_estimators': 60}
0.8
(b'LoyalCH', 0.44873561)
(b'PriceDiff', 0.07308436)
(b'StoreID', 0.07110346)
(b'ListPriceD', 0.07060679)
(b'STORE', 0.06812638)
